In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from Twitter_bot_detection_713.data_prep import tweet_df_cleaner, user_df_cleaner, get_final_tweet_data

2021-11-29 21:45:58.646152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 21:45:58.646238: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
user_df = pd.read_csv('../raw_data/users_data.csv',
                          sep='\t',
                          lineterminator='\n')

In [3]:
df = user_df_cleaner(user_df)

In [4]:
df

,author_id,username,user_display_name,user_desc,user_created_at,user_verified,user_private,user_followers_cnt,user_following_cnt,user_tweet_count,user_list_count,target
0,787405734442958848,best_in_dumbest,The Best In Dumbest,"Blame @xaiax, Inspired by @MakingInvisible, us...",2016-10-15 21:32:11+00:00,False,False,1848,5,14542,0,bot
1,796216118331310080,CJRubinPhoto,CJ Rubin,Photographing the American West since 1980. I ...,2016-11-09 05:01:30+00:00,False,False,812,845,251,5,human
2,875949740503859204,SVGEGENT,🔥𝖙𝖍𝖊 𝖘𝖆𝖛𝖆𝖌𝖊 𝖌𝖊𝖓𝖙𝖑𝖊𝖒𝖆𝖓🔥,Part savage all gentleman ...,2017-06-17 05:34:27+00:00,False,False,182,885,1288,2,human
3,756119643622735875,TinkerVHECHK5,Danielle Tinker,Wife.Godmother.Friend.Feline Fanatic! Principa...,2016-07-21 13:32:25+00:00,False,False,657,747,1920,2,human
4,464781334,JoleonLescott,Joleon Lescott,England U21 Assistant Coach | @pumafootball Am...,2012-01-15 16:32:35+00:00,True,False,739915,120,4459,1800,human
...,...,...,...,...,...,...,...,...,...,...,...,...
36229,63963107,butchsekab,Rosemary Pearman nee Bakes,"Unemployed! Retail Assistant, Stocktaker, Dry ...",2009-08-08 13:11:15+00:00,False,False,152,1219,487,3,human
36230,1064042478,Mathieu_Era,Tyrann Mathieu,FOLLOW ME ON IG.,2013-01-05 21:35:37+00:00,True,False,1237990,799,27326,1475,human
36231,1089732602,SDalmaoficial,Sergio Dalma,Bienvenid@s al Twitter Oficial de Sergio Dalma...,2013-01-14 18:05:54+00:00,True,False,85615,189,6640,270,human
36232,815529979,VinceYearly,Mr. Vance,Just a good guy wrapped up in a bad system.\nA...,2012-09-10 16:12:02+00:00,False,False,137,166,1040,3,human


In [11]:
X = df[['user_verified', 'user_private', 'user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']]
y = df['target'].apply(lambda x: 1 if x == 'bot' else 0)

In [12]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
df[['user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']].describe()

In [ ]:
df.corr()

## Duplicates & Missing Values

In [ ]:
df.duplicated().sum()

NO DUPLICATES

In [ ]:
(df.isnull().sum() / len(df)) 

### Imputing missing values

In [7]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy='constant', fill_value='no user_desc')

imputer.fit(df[['user_desc']])

df['user_desc'] = imputer.transform(df[['user_desc']])

In [ ]:
(df.isnull().sum() / len(df)) * 100

## Outliers - Plotting

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.boxplot(data=df, x='user_followers_cnt')

In [ ]:
sns.histplot(data=df, x='user_tweet_count')
plt.xlim(0,2)
plt.ylim(0,10000)

## Robust-Scaler for num_vars

In [ ]:

r_scaler = RobustScaler()

num_features = df[['user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']]

r_scaler.fit(num_features)
df[['user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']] = r_scaler.transform(num_features)

In [ ]:
df.head()

## Labeling 'bool' and target 

In [ ]:
df['user_private'].value_counts()

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

In [ ]:
label.fit(df['target'])

df['target'] = label.transform(df['target'])

In [ ]:
df.head()

In [ ]:
df['user_verified'] = df['user_verified'].map(lambda x: 1 if x == True else 0)

In [ ]:
df['user_private'] = df['user_private'].map(lambda x: 1 if x == True else 0)

In [ ]:
df['target'] = df['target'].map(lambda x: 1 if x == 'bot' else 0)

In [ ]:
df['user_private'].value_counts()

In [ ]:
df.head()

## Correlation & Heatmap

In [ ]:
df[['user_verified', 'user_private', 'user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']].corr()

In [ ]:
corr = df[['user_verified', 'user_private', 'user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count']].corr()
sns.heatmap(corr, cmap="YlGnBu")

## Pipeline

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [13]:


preprocessor = ColumnTransformer([
    ('num_tr', RobustScaler(), ['user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count'])    
])

pipe = Pipeline([
                ('preprocessor', preprocessor),
                ('log_reg', LogisticRegression())     
])

df_transformed = pipe.fit(X_train, y_train)



In [ ]:
preprocessor = ColumnTransformer([
    #('imputer', SimpleImputer(strategy='constant', fill_value='no user_desc'), ['user_desc']),
    ('num_tr', RobustScaler(), ['user_followers_cnt', 'user_following_cnt', 'user_tweet_count', 'user_list_count'])
    #('cat_tr', LabelEncoder(), ['user_verified', 'user_private'])
])

In [ ]:
X_train.dtypes

In [ ]:
x_preproc = preprocessor.fit_transform(X_train)

In [ ]:
x_preproc